## imports

In [ ]:
import pickle
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import cv2
import random

# Part 1

## load dataset

In [ ]:
def load_dataset():
  image_pickle_file_path = 'images.pkl'
  label_pickle_file_path = 'label.pkl'

  with open(image_pickle_file_path, 'rb') as file:
    images = pickle.load(file)

  with open(label_pickle_file_path, 'rb') as file:
    labels = pickle.load(file)

  # images = images.reshape(images.shape[0], -1)

  return images, labels

In [ ]:
images, labels = load_dataset()
random_indices = np.random.choice(560, size=15, replace=False)
# Extract the randomly selected values
random_values = images[random_indices]

from dataclasses import dataclass, field


@dataclass
class Images:
    image: np.ndarray = None
    original_features = []
    features = []
    clusters = None




## Proccess on images
- Extract features
- normalize features
- make features a single dimnetion vector

In [ ]:
def extract_features(image):
    features = []
    original_features = []

    # height, width, _ = image.shape
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # For each pixel in the image
    for i in range(img_hsv.shape[0]):
        for j in range(img_hsv.shape[1]):
            pixel = img_hsv[i, j]
            h, s, v = pixel
            x, y = i, j

            original_features.append((h,s,v,x,y))

            h_weight = 10
            s_weight = 1
            v_weight = 1
            x_weight = 0
            y_weight = 0
            h, s, v = h * h_weight, s * s_weight, v * v_weight
            x, y = x * x_weight, y * y_weight

            features.append((h, s, v, x, y))

    return features, original_features

In [ ]:
def normalize_features(features):
    features_array = np.array(features)
    scaler = StandardScaler()
    normalized_features_array = scaler.fit_transform(features_array)
    normalized_features = normalized_features_array.tolist()
    return normalized_features

In [ ]:
new_features = []

for img in random_values:
    features, original_features = extract_features(img)
    new_features.append(features)

In [ ]:
# normalize features
for i, fimg in enumerate(new_features):
    new_features[i] = normalize_features(fimg)

## clustering

### k-means

In [ ]:
kmeans = KMeans(n_clusters=3)
new_clusters = []
for nf in new_features:
    clusters = kmeans.fit_predict(nf)
    new_clusters.append(clusters)

In [ ]:
for i, nc in enumerate(new_clusters):
    new_clusters[i] = nc.reshape(random_values.shape[1], random_values.shape[2])

In [ ]:
image0_clusters = new_clusters[0]
image0_clusters

## display clustering

In [ ]:
new_cluster_mean = []
for nc in new_clusters:
    clusters_mean = cluster_mean_value(nc)
    new_cluster_mean.append(clusters_mean)
# clusters_mean.reshape(clusters_mean.shape[0]*clusters_mean.shape[1])

## extracting clusters features for each image sepratedly

### mean color for each cluster in each image

In [ ]:
def mean_find_matching_triads(array1, array2):   
    unique_values = np.unique(array1)
    
    result = []

    for value in unique_values:
        indexes = np.where(array1 == value)[0]
        triads = array2[indexes, :].tolist()
        result.append(triads)

    return result

In [ ]:
def calculate_mean(data):
    data_array = np.array(data)
    mean_values = np.mean(data_array, axis=0)
    mean_values = mean_values.astype(int)
    mean_values_list = mean_values.tolist()
    return mean_values_list

In [ ]:
def mean_forward_each_image_(new_cls, random_values):

    all_images_mean_values = []
    for i, image_cls in enumerate(new_cls):
        one_image_clusters_mean_values = []
        reshaped_image_cls = image_cls.reshape(image_cls.shape[0] * image_cls.shape[1])
        reshaped_random_value = random_values[i].reshape(random_values.shape[1] * random_values.shape[2], random_values.shape[3])

        similar_clusters_pixels = mean_find_matching_triads(reshaped_image_cls, reshaped_random_value)

        for smp in similar_clusters_pixels:
            one_image_clusters_mean_values.append(calculate_mean(smp))

        all_images_mean_values.append(one_image_clusters_mean_values)
    return all_images_mean_values

In [ ]:
mean_forward_each_image = mean_forward_each_image_(new_clusters, random_values)


In [ ]:
image0_clusters.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import colorsys

# Given data
pixel_values = image0_clusters
hsv_means = mean_forward_each_image[0]

# Reshape the pixel values to match the image dimensions
# image_pixels = pixel_values.reshape(3, 3)

# Create an empty array to hold the HSV values for each pixel
rgb_image = np.zeros((pixel_values.shape[0], pixel_values.shape[1], 3), dtype=np.uint8)

# Assign the corresponding HSV values to each pixel
for i in range(pixel_values.shape[0]):
    for j in range(pixel_values.shape[1]):
        value = pixel_values[i, j]
        hsv = hsv_means[value - 1]
        rgb = colorsys.hsv_to_rgb(hsv[0] / 360, hsv[1] / 100, hsv[2] / 100)
        rgb_image[i, j] = np.array(rgb) * 150  # Convert RGB values to 0-255 range

# Display the image with corresponding RGB values using cv2.imshow
cv2.imshow('Image with Corresponding RGB Values', cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()



In [ ]:
print(mean_forward_each_image[0])

### variance color for each cluster in each image

In [ ]:
def var_find_matching_triads(array1, array2):   

    unique_values = np.unique(array1)

    result = []

    for value in unique_values:
        indexes = np.where(array1 == value)[0]
        triads = array2[indexes, :].tolist()
        result.append(triads)

    return result

In [ ]:
def calculate_var(data):
    data_array = np.array(data)
    var_values = np.var(data_array, axis=0)
    var_values = var_values.astype(int)
    var_values_list = var_values.tolist()
    return var_values_list



In [ ]:
def var_forward_each_image(clusters, random_values):
    similar_clusters_pixels = []

    var_values = []
    reshaped_clusters = clusters.reshape(clusters.shape[0], clusters.shape[1] * clusters.shape[2])
    reshaped_random_values = random_values.reshape(random_values.shape[0],
                                                    random_values.shape[1] * random_values.shape[2],
                                                    random_values.shape[3])
    for i in range(random_values.shape[0]):
        similar_clusters_pixels = var_find_matching_triads(reshaped_clusters[i],
                                                        reshaped_random_values[i])
        each_cluster_var_values = []
        for j in range(len(similar_clusters_pixels)):
            each_cluster_var_values.append(calculate_var(similar_clusters_pixels[j]))

        var_values.append(each_cluster_var_values)

    return var_values


In [ ]:
var_color_each_cluster_each_image = var_forward_each_image(clusters, random_values)
for i in var_color_each_cluster_each_image:
    print(len(i) != 10)

### size of clusters in each image

In [ ]:
def calculate_size(clusters):
    size_each_cluster_each_image = []

    for i in range(clusters.shape[0]):
        unique_clusters, counts = np.unique(clusters[i], return_counts=True)
        each_clusters_size = counts
        size_each_cluster_each_image.append(each_clusters_size)

    # size_each_cluster_each_image = np.array(size_each_cluster_each_image)
    return size_each_cluster_each_image

In [ ]:
size_each_cluster_each_image = calculate_size(clusters)
size_each_cluster_each_image

# Part 2

## create clusters feature vectors 

In [ ]:
def create_clusters_vector(clusters_mean, clusters_variance):
    result = []
    for mean, variance in zip(clusters_mean, clusters_variance):
        if (len(mean) != 10 or len(variance) != 10):
            print("Jib")
            continue
        for i in range(10):
            result.append([*mean[i], *variance[i]])
    return result

In [ ]:
def flatten_nested_list(nested_list):
    return [[item for sublist in lst for item in (sublist if isinstance(sublist, list) else [sublist])] for lst in nested_list]

In [ ]:
clusters_features = create_clusters_vector(mean_color_each_cluster_each_image, var_color_each_cluster_each_image)
# clusters_features = flatten_nested_list(clusters_features)
clusters_featuresss = np.array(clusters_features)
clusters_featuresss

## clustering clusters

In [ ]:
kmeans_2 = KMeans(n_clusters=3)
clusters_features_flat = [np.concatenate([np.array(item[:-1]), item[-1]]) for item in clusters_features]
clusters_2 = kmeans.fit_predict(clusters_features_flat)
# clusters_2 = kmeans.fit_predict(clusters_features)
clusters_2

## Diaplay clustered cluster

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
pca = PCA(2)
data_transformed = pca.fit_transform(clusters_features_flat)

plt.scatter(data_transformed[:, 0], data_transformed[:, 1], c=clusters_2)
plt.show()
